Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-138850
aml-quickstarts-138850
southcentralus
aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-experiment-udacity'
project_folder = './pipeline-bankmarketing-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-experiment-udacity,quick-starts-ws-138850,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-udacity"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Bank_marketing"
description_text = "Bank Marketing DataSet"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Azure/MachineLearningNotebooks/master/how-to-use-azureml/automated-machine-learning/forecasting-bike-share/bike-no.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [7]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [9]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [10]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [11]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [12]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [723b4fa2][a2ff25d4-2eda-4f99-82a7-150025480aa2], (This step will run and generate new outputs)
Submitted PipelineRun 2c7fc037-23fd-483c-8de0-29920ab3457a
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-udacity/runs/2c7fc037-23fd-483c-8de0-29920ab3457a?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-138850/workspaces/quick-starts-ws-138850


In [13]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [14]:
pipeline_run.wait_for_completion()

PipelineRunId: 2c7fc037-23fd-483c-8de0-29920ab3457a
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-udacity/runs/2c7fc037-23fd-483c-8de0-29920ab3457a?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-138850/workspaces/quick-starts-ws-138850
PipelineRun Status: Running


StepRunId: 2237ae0a-db52-42cf-88f3-dbdf51ba2f85
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-udacity/runs/2237ae0a-db52-42cf-88f3-dbdf51ba2f85?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-138850/workspaces/quick-starts-ws-138850
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '2237ae0a-db52-42cf-88f3-dbdf51ba2f85', 'target': 'compute-udacity', 'status': 'Completed', 'startTimeUtc': '2021-02-14T01:56:50.835146Z', 'endTimeUtc': '2021-02-14T02:32:05.172484Z', 'properties': {'Conte

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [15]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/2237ae0a-db52-42cf-88f3-dbdf51ba2f85/metrics_data, 1 files out of an estimated total of 1


In [16]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_1,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_6,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_21,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_5,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_33,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_32,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_43,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_48,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_10,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_19,...,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_14,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_16,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_23,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_37,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_31,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_35,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_38,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_40,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_4,2237ae0a-db52-42cf-88f3-dbdf51ba2f85_55
precision_score_micro,[0.9071320182094081],[0.9001517450682853],[0.9101669195751139],[0.8977238239757208],[0.9116843702579667],[0.9059180576631259],[0.844309559939302],[0.9089529590288316],[0.8109256449165402],[0.9125948406676783],...,[0.8928679817905918],[0.7893778452200303],[0.9004552352048558],[0.9119878603945372],[0.9089529590288316],[0.8880121396054628],[0.8880121396054628],[0.9119878603945372],[0.8880121396054628],[0.9095599393019727]
f1_score_weighted,[0.9021127651963996],[0.8730666405599883],[0.9026632441364442],[0.8734704046383025],[0.9084143013294615],[0.8999468147095336],[0.8664971741551346],[0.9064064693565229],[0.8368248473608529],[0.9086613440609772],...,[0.8517088092148907],[0.8221303205128497],[0.8719631449552753],[0.9081566096944386],[0.9015060382046648],[0.8353395018439429],[0.8353395018439429],[0.9084131426940711],[0.8353395018439429],[0.9038198412369066]
norm_macro_recall,[0.43834549418631563],[0.1747124648279974],[0.4109757023749321],[0.18855620203502088],[0.48846895509283184],[0.41803233138278073],[0.7133669354465246],[0.49012961079713335],[0.509994498441225],[0.4800211911893555],...,[0.0670430696104638],[0.5354628255783584],[0.16558117392520466],[0.4793376641900391],[0.40724038477568647],[0.0],[0.0],[0.4840741913912645],[0.0],[0.43634307498235625]
average_precision_score_macro,[0.8065229883244922],[0.7485206211176303],[0.7998321444303222],[0.7319927178406187],[0.8120523754838133],[0.7964818798092093],[0.7548023581323982],[0.8160119279976998],[0.6893459218106075],[0.8126929119384294],...,[0.7243940093625936],[0.7225261606701461],[0.787108791806625],[0.8071742282912644],[0.7992078386060956],[0.5],[0.7996165142711693],[0.8194395466063031],[0.7283140353668054],[0.8103790221049284]
recall_score_weighted,[0.9071320182094081],[0.9001517450682853],[0.9101669195751139],[0.8977238239757208],[0.9116843702579667],[0.9059180576631259],[0.844309559939302],[0.9089529590288316],[0.8109256449165402],[0.9125948406676783],...,[0.8928679817905918],[0.7893778452200303],[0.9004552352048558],[0.9119878603945372],[0.9089529590288316],[0.8880121396054628],[0.8880121396054628],[0.9119878603945372],[0.8880121396054628],[0.9095599393019727]
recall_score_micro,[0.9071320182094081],[0.9001517450682853],[0.9101669195751139],[0.8977238239757208],[0.9116843702579667],[0.9059180576631259],[0.844309559939302],[0.9089529590288316],[0.8109256449165402],[0.9125948406676783],...,[0.8928679817905918],[0.7893778452200303],[0.9004552352048558],[0.9119878603945372],[0.9089529590288316],[0.8880121396054628],[0.8880121396054628],[0.9119878603945372],[0.8880121396054628],[0.9095599393019727]
AUC_macro,[0.9392346349984347],[0.8957950122905194],[0.9342679499932389],[0.9025728586062347],[0.9432061306258996],[0.9336923239362265],[0.9229346462979326],[0.9424512871239443],[0.8264901907392279],[0.943998021661693],...,[0.8700279894118148],[0.8618719748373149],[0.9304904908242521],[0.9407813695361835],[0.9355493315698706],[0.5],[0.9333274057279193],[0.9446315344903278],[0.8945381747050554],[0.9419488299462626]
average_precision_score_weighted,[0.9505970434373063],[0.9325737036930096],[0.9486146431518476],[0.9294231352417048],[0.9522998164221196],[0.9475102670049403],[0.936

### Retrieve the Best Model

In [17]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/2237ae0a-db52-42cf-88f3-dbdf51ba2f85/model_data, 1 files out of an estimated total of 1


In [18]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [19]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('42',
                                             Pipeline(memory=None,
                                                      steps=[('standardscalerwrapper',
                                                              <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7fdd38c19d68>),
                                                             ('xgboostclassifier',
                                                              XGBoostClassifier(ba

## Test the Model

### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [20]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

In [21]:
X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,371,1,999,1,failure,-1.80,92.89,-46.20,1.30,5099.10
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,285,2,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,52,1,999,1,failure,-1.80,92.89,-46.20,1.31,5099.10
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,355,4,999,0,nonexistent,1.40,94.47,-41.80,4.97,5228.10
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,189,2,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,116,1,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,69,7,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,135,4,999,1,failure,-1.80,92.89,-46.20,1.27,5099.10
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,386,1,999,0,nonexistent,-1.80,93.08,-47.10,1.41,5099.10


In [22]:
y_test

0        no
1        no
2        no
3        no
4        no
         ..
32945    no
32946    no
32947    no
32948    no
32949    no
Name: y, Length: 32950, dtype: object

### Testing Our Best Fitted Model
We will use confusion matrix to see how our model works.

In [23]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [25]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,28762,496
1,1079,2613


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [27]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
Bankmarketing Train,b2b8835e-427d-4da9-b998-3fa9bb286cb0,Active,REST Endpoint


In [28]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [29]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bank-rest-endpoint"}
                        )

In [30]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  5878e5af-cd3f-40a0-9464-535d7562d241


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [32]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["ml-experiment-udacity"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …